**Task F — Free-text Weather Extraction & Unit Normalization**

**Description:**
Given a free-text weather report file with heterogeneous formats and possibly mixed languages, extract `date`, `max_temp`, `min_temp`, `humidity`, `precipitation` using robust regexes. Normalize units (e.g., Fahrenheit → Celsius), handle `,` vs `.` decimal separators, and produce `extracted_weather_data.csv`.

**Deliverables:**

* `extracted_weather_data.csv` with headers `Date,Max Temperature,Min Temperature,Humidity,Precipitation`
* A short patterns file `patterns.txt` documenting regexes used

**Expected CSV row (example):**

```
2024-08-18,32.9,22.5,65,0.0
```

**Hints:**

* Prepare multiple regex patterns to match different log styles and test them in order.
* Implement `normalize_number()` to handle `32,5` → `32.5`.
* Implement a simple F→C conversion and heuristics to detect Fahrenheit (e.g., values > 60 likely F or explicit `F` marker).

**Run-and-paste (live check):**
Run the parsing cell for a single sample line and paste the parsed dictionary into the chat.

In [ ]:
# Task F - Free-text Weather Extraction (BUGGY VERSION)
import re
import pandas as pd
from datetime import datetime

def create_sample_weather_logs():
    """Create sample free-text weather logs with various formats"""
    weather_logs = """2024-08-18: Máximo 87°F, mínimo 72°F, humedad 65%, precipitación 0,0mm
August 19, 2024 - High: 91F Low: 75F Humidity: 72% Rain: 2.5mm
20/08/2024 Max temp: 32,9°C Min: 22,5°C Humid: 58% Precip: 0.0
2024-08-21 Maximum temperature 89°F minimum 74°F humidity 68% precipitation 1,2 millimeters
Aug 22 '24: TOP 35.1°C BOTTOM 25.7°C H=63% P=0mm
23.08.2024 - Høy: 88°F Lav: 71°F Fuktighet: 59% Nedbør: 0,5mm
24-Aug-2024: Max 33.8C Min 24.1C Humidity 61% Rain 0.0mm
2024/08/25 High temp 90°F Low temp 73°F RH 66% Precipitation 3,1mm"""
    
    with open('weather_logs.txt', 'w', encoding='utf-8') as f:
        f.write(weather_logs)
    
    print("Sample weather logs created: weather_logs.txt")

def normalize_number_buggy(text):
    """
    Normalize number with decimal separator
    BUG: Not handling all cases properly
    """
    if not text:
        return None
    
    # BUG 1: Not handling spaces in numbers
    text = text.strip()
    
    # BUG 2: Simple comma to dot replacement without validation
    text = text.replace(',', '.')
    
    try:
        return float(text)
    except ValueError:
        return None

def fahrenheit_to_celsius_buggy(temp):
    """
    Convert Fahrenheit to Celsius
    BUG: Contains calculation error
    """
    if temp is None:
        return None
    
    # BUG 3: Wrong conversion formula
    return (temp - 32) * 5 / 8  # Should be 5/9

def detect_fahrenheit_buggy(temp_text, temp_value):
    """
    Detect if temperature is in Fahrenheit
    BUG: Poor detection logic
    """
    if not temp_text or temp_value is None:
        return False
    
    # BUG 4: Only checking for 'F' but not '°F'
    if 'F' in temp_text.upper():
        return True
    
    # BUG 5: Wrong threshold for Fahrenheit detection
    if temp_value > 50:  # Should be around 60
        return True
    
    return False

def extract_weather_data_buggy(text):
    """
    Extract weather data from free text
    BUG: Contains multiple regex and parsing bugs
    """
    # BUG 6: Limited date patterns
    date_patterns = [
        r'(\d{4}-\d{2}-\d{2})',
        r'(\d{2}/\d{2}/\d{4})',
        r'(\w+ \d{1,2}, \d{4})'
    ]
    
    # BUG 7: Incomplete temperature patterns
    temp_patterns = [
        r'(?:max|máximo|high|høy|top).*?(\d+(?:[,.]\d+)?)\s*°?([CF])?',
        r'(?:min|mínimo|low|lav|bottom).*?(\d+(?:[,.]\d+)?)\s*°?([CF])?'
    ]
    
    # BUG 8: Basic humidity pattern missing variations
    humidity_pattern = r'(?:humidity|humedad|humid|fuktighet|h).*?(\d+(?:[,.]\d+)?)'
    
    # BUG 9: Incomplete precipitation patterns
    precip_patterns = [
        r'(?:rain|precipitation|precipitación|precip|nedbør|p).*?(\d+(?:[,.]\d+)?)',
    ]
    
    extracted = {}
    
    # Extract date
    date_found = False
    for pattern in date_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            extracted['date'] = match.group(1)
            date_found = True
            break
    
    # BUG 10: No fallback for missing date
    if not date_found:
        extracted['date'] = None
    
    # Extract temperatures
    max_temp = None
    min_temp = None
    
    # BUG 11: Only checking first pattern
    for pattern in temp_patterns[:1]:  # Should check all patterns
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            for match in matches:
                temp_str, unit = match
                temp_val = normalize_number_buggy(temp_str)
                
                if detect_fahrenheit_buggy(temp_str + unit, temp_val):
                    temp_val = fahrenheit_to_celsius_buggy(temp_val)
                
                # BUG 12: Poor max/min assignment logic
                if max_temp is None:
                    max_temp = temp_val
                else:
                    min_temp = temp_val
    
    extracted['max_temp'] = max_temp
    extracted['min_temp'] = min_temp
    
    # Extract humidity
    humidity_match = re.search(humidity_pattern, text, re.IGNORECASE)
    if humidity_match:
        extracted['humidity'] = normalize_number_buggy(humidity_match.group(1))
    else:
        extracted['humidity'] = None
    
    # Extract precipitation
    precipitation = None
    for pattern in precip_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            precipitation = normalize_number_buggy(match.group(1))
            break
    
    extracted['precipitation'] = precipitation
    
    return extracted

def save_patterns_buggy():
    """Save regex patterns to file"""
    patterns = """Date Patterns:
(\d{4}-\d{2}-\d{2})
(\d{2}/\d{2}/\d{4})
(\w+ \d{1,2}, \d{4})

Temperature Patterns:
(?:max|máximo|high|høy|top).*?(\d+(?:[,.]\d+)?)\s*°?([CF])?

Humidity Pattern:
(?:humidity|humedad|humid|fuktighet|h).*?(\d+(?:[,.]\d+)?)

Precipitation Pattern:
(?:rain|precipitation|precipitación|precip|nedbør|p).*?(\d+(?:[,.]\d+)?)"""
    
    with open('patterns_buggy.txt', 'w', encoding='utf-8') as f:
        f.write(patterns)

def main_weather_extraction_buggy():
    """Main function for weather extraction (buggy version)"""
    print("=== Task F: Free-text Weather Extraction (BUGGY VERSION) ===")
    
    # Create sample data
    create_sample_weather_logs()
    
    # Read and process logs
    with open('weather_logs.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    extracted_data = []
    
    print("Processing weather logs...")
    for i, line in enumerate(lines):
        line = line.strip()
        if line:
            print(f"Line {i+1}: {line}")
            result = extract_weather_data_buggy(line)
            print(f"Extracted: {result}")
            extracted_data.append(result)
            print()
    
    # Save to CSV
    df = pd.DataFrame(extracted_data)
    df.columns = ['Date', 'Max Temperature', 'Min Temperature', 'Humidity', 'Precipitation']
    df.to_csv('extracted_weather_data_buggy.csv', index=False)
    
    # Save patterns
    save_patterns_buggy()
    
    print("Results saved to extracted_weather_data_buggy.csv and patterns_buggy.txt")
    
    # Show first extraction for paste-and-check
    if extracted_data:
        print("First extraction result:")
        print(extracted_data[0])
    
    return extracted_data

# Run buggy version
if __name__ == "__main__":
    result_buggy = main_weather_extraction_buggy()

In [ ]:
# Task F - Free-text Weather Extraction (FIXED VERSION)
import re
import pandas as pd
from datetime import datetime

def normalize_number_fixed(text):
    """
    Normalize number with decimal separator - FIXED
    FIXES: Proper handling of spaces, validation, and decimal conversion
    """
    if not text:
        return None
    
    # FIX 1: Remove spaces and clean text
    text = re.sub(r'\s+', '', text.strip())
    
    # FIX 2: Handle both comma and dot as decimal separator
    # Check if it's likely a decimal (comma/dot followed by 1-2 digits)
    if re.match(r'^\d+[,.]\d{1,2}$', text):
        text = text.replace(',', '.')
    elif ',' in text and '.' not in text:
        # Handle cases like "1,234" (could be decimal or thousands separator)
        parts = text.split(',')
        if len(parts) == 2 and len(parts[1]) <= 2:
            text = text.replace(',', '.')
    
    # Remove any non-digit characters except decimal point
    text = re.sub(r'[^\d.]', '', text)
    
    try:
        return float(text)
    except ValueError:
        return None

def fahrenheit_to_celsius_fixed(temp):
    """
    Convert Fahrenheit to Celsius - FIXED
    FIX 3: Correct conversion formula
    """
    if temp is None:
        return None
    
    # FIXED: Correct formula (F - 32) * 5/9
    return round((temp - 32) * 5 / 9, 1)

def detect_fahrenheit_fixed(temp_text, temp_value):
    """
    Detect if temperature is in Fahrenheit - FIXED
    FIX 4-5: Better detection logic with multiple indicators
    """
    if not temp_text or temp_value is None:
        return False
    
    # FIX 4: Check for various Fahrenheit indicators
    fahrenheit_indicators = ['°F', 'F', 'fahrenheit']
    for indicator in fahrenheit_indicators:
        if indicator in temp_text.upper():
            return True
    
    # FIX 5: Better threshold - temperatures above 60 likely Fahrenheit
    # Also consider context - if no unit specified and temp > 40, likely Fahrenheit
    if temp_value > 60:
        return True
    elif temp_value > 40 and not any(c in temp_text.upper() for c in ['°C', 'C', 'CELSIUS']):
        return True
    
    return False

def extract_date_fixed(text):
    """Extract and normalize date from text - FIXED"""
    # FIX 6: Comprehensive date patterns
    date_patterns = [
        (r'(\d{4}-\d{2}-\d{2})', '%Y-%m-%d'),  # 2024-08-18
        (r'(\d{2}/\d{2}/\d{4})', '%d/%m/%Y'),  # 20/08/2024
        (r'(\d{4}/\d{2}/\d{2})', '%Y/%m/%d'),  # 2024/08/25
        (r'(\w+\s+\d{1,2},?\s+\d{4})', None),  # August 19, 2024
        (r'(\d{1,2}-\w+-\d{4})', None),        # 24-Aug-2024
        (r'(\d{2}\.\d{2}\.\d{4})', '%d.%m.%Y') # 23.08.2024
    ]
    
    for pattern, date_format in date_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            date_str = match.group(1)
            
            if date_format:
                try:
                    date_obj = datetime.strptime(date_str, date_format)
                    return date_obj.strftime('%Y-%m-%d')
                except ValueError:
                    continue
            else:
                # Handle special formats
                try:
                    # Try parsing with dateutil or manual conversion
                    if 'Aug' in date_str or 'August' in date_str:
                        # Convert month names
                        date_str = date_str.replace('Aug ', 'August ')
                        date_str = date_str.replace("'24", " 2024")
                        date_obj = datetime.strptime(date_str, '%B %d, %Y')
                        return date_obj.strftime('%Y-%m-%d')
                    elif '-' in date_str and any(month in date_str for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']):
                        # Handle 24-Aug-2024 format
                        parts = date_str.split('-')
                        if len(parts) == 3:
                            day, month, year = parts
                            date_obj = datetime.strptime(f"{day}-{month}-{year}", '%d-%b-%Y')
                            return date_obj.strftime('%Y-%m-%d')
                except ValueError:
                    continue
    
    return None

def extract_temperatures_fixed(text):
    """Extract max and min temperatures - FIXED"""
    # FIX 7: Comprehensive temperature patterns
    temp_patterns = [
        # Pattern for max/high temperatures
        r'(?:max(?:imum)?|máximo|high|høy|top)\s*(?:temp(?:erature)?)?\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*°?([CF]?)',
        # Pattern for min/low temperatures  
        r'(?:min(?:imum)?|mínimo|low|lav|bottom)\s*(?:temp(?:erature)?)?\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*°?([CF]?)',
        # General temperature pattern
        r'(\d+(?:[,.]\d+)?)\s*°([CF]?)'
    ]
    
    max_temp = None
    min_temp = None
    
    # Look for explicit max/min indicators
    max_patterns = [
        r'(?:max(?:imum)?|máximo|high|høy|top)\s*(?:temp(?:erature)?)?\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*°?([CF]?)',
    ]
    
    min_patterns = [
        r'(?:min(?:imum)?|mínimo|low|lav|bottom)\s*(?:temp(?:erature)?)?\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*°?([CF]?)',
    ]
    
    # Extract max temperature
    for pattern in max_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            temp_str, unit = match.groups()
            temp_val = normalize_number_fixed(temp_str)
            
            if detect_fahrenheit_fixed(temp_str + (unit or ''), temp_val):
                temp_val = fahrenheit_to_celsius_fixed(temp_val)
            
            max_temp = temp_val
            break
    
    # Extract min temperature
    for pattern in min_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            temp_str, unit = match.groups()
            temp_val = normalize_number_fixed(temp_str)
            
            if detect_fahrenheit_fixed(temp_str + (unit or ''), temp_val):
                temp_val = fahrenheit_to_celsius_fixed(temp_val)
            
            min_temp = temp_val
            break
    
    return max_temp, min_temp

def extract_weather_data_fixed(text):
    """
    Extract weather data from free text - FIXED
    FIXES: All bugs from buggy version addressed
    """
    extracted = {}
    
    # FIX 10: Extract date with fallback
    extracted['date'] = extract_date_fixed(text)
    if not extracted['date']:
        print(f"Warning: Could not extract date from: {text[:50]}...")
    
    # FIX 11: Extract temperatures properly
    max_temp, min_temp = extract_temperatures_fixed(text)
    extracted['max_temp'] = max_temp
    extracted['min_temp'] = min_temp
    
    # FIX 8: Enhanced humidity pattern
    humidity_patterns = [
        r'(?:humidity|humedad|humid|fuktighet|rh|h)\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*%?',
    ]
    
    humidity = None
    for pattern in humidity_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            humidity = normalize_number_fixed(match.group(1))
            # Ensure humidity is reasonable (0-100%)
            if humidity and 0 <= humidity <= 100:
                break
            else:
                humidity = None
    
    extracted['humidity'] = humidity
    
    # FIX 9: Comprehensive precipitation patterns
    precip_patterns = [
        r'(?:rain|precipitation|precipitación|precip|nedbør|p)\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*(?:mm|millimeters?)?',
    ]
    
    precipitation = None
    for pattern in precip_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            precipitation = normalize_number_fixed(match.group(1))
            break
    
    extracted['precipitation'] = precipitation
    
    return extracted

def save_patterns_fixed():
    """Save comprehensive regex patterns to file"""
    patterns = """Date Patterns (FIXED):
(\d{4}-\d{2}-\d{2}) - ISO format
(\d{2}/\d{2}/\d{4}) - DD/MM/YYYY
(\d{4}/\d{2}/\d{2}) - YYYY/MM/DD
(\w+\s+\d{1,2},?\s+\d{4}) - Month Day, Year
(\d{1,2}-\w+-\d{4}) - DD-Mon-YYYY
(\d{2}\.\d{2}\.\d{4}) - DD.MM.YYYY

Temperature Patterns (FIXED):
(?:max(?:imum)?|máximo|high|høy|top)\s*(?:temp(?:erature)?)?\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*°?([CF]?)
(?:min(?:imum)?|mínimo|low|lav|bottom)\s*(?:temp(?:erature)?)?\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*°?([CF]?)

Humidity Pattern (FIXED):
(?:humidity|humedad|humid|fuktighet|rh|h)\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*%?

Precipitation Pattern (FIXED):
(?:rain|precipitation|precipitación|precip|nedbør|p)\s*[:=]?\s*(\d+(?:[,.]\d+)?)\s*(?:mm|millimeters?)?

Unit Conversion:
Fahrenheit to Celsius: (F - 32) * 5/9
Detection: Look for °F, F indicators or temp > 60"""
    
    with open('patterns_fixed.txt', 'w', encoding='utf-8') as f:
        f.write(patterns)

def validate_extraction_fixed(extracted_data):
    """Validate extracted data and report issues"""
    print("Validation Report:")
    
    total_lines = len(extracted_data)
    valid_dates = sum(1 for item in extracted_data if item['date'])
    valid_max_temps = sum(1 for item in extracted_data if item['max_temp'] is not None)
    valid_min_temps = sum(1 for item in extracted_data if item['min_temp'] is not None)
    valid_humidity = sum(1 for item in extracted_data if item['humidity'] is not None)
    valid_precipitation = sum(1 for item in extracted_data if item['precipitation'] is not None)
    
    print(f"  Total lines processed: {total_lines}")
    print(f"  Valid dates extracted: {valid_dates}/{total_lines}")
    print(f"  Valid max temperatures: {valid_max_temps}/{total_lines}")
    print(f"  Valid min temperatures: {valid_min_temps}/{total_lines}")
    print(f"  Valid humidity values: {valid_humidity}/{total_lines}")
    print(f"  Valid precipitation values: {valid_precipitation}/{total_lines}")

def main_weather_extraction_fixed():
    """Main function for weather extraction (fixed version)"""
    print("=== Task F: Free-text Weather Extraction (FIXED VERSION) ===")
    
    # Use existing sample data
    try:
        with open('weather_logs.txt', 'r', encoding='utf-8') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print("Creating sample data...")
        create_sample_weather_logs()
        with open('weather_logs.txt', 'r', encoding='utf-8') as f:
            lines = f.readlines()
    
    extracted_data = []
    
    print("Processing weather logs with fixed extraction...")
    for i, line in enumerate(lines):
        line = line.strip()
        if line:
            print(f"Line {i+1}: {line}")
            result = extract_weather_data_fixed(line)
            print(f"Extracted: {result}")
            extracted_data.append(result)
            print()
    
    # Validate results
    validate_extraction_fixed(extracted_data)
    
    # Save to CSV
    df = pd.DataFrame(extracted_data)
    df.columns = ['Date', 'Max Temperature', 'Min Temperature', 'Humidity', 'Precipitation']
    
    # Handle missing values
    df = df.fillna('')
    df.to_csv('extracted_weather_data_fixed.csv', index=False)
    
    # Save patterns
    save_patterns_fixed()
    
    print("Results saved to extracted_weather_data_fixed.csv and patterns_fixed.txt")
    
    # Show first extraction for paste-and-check
    if extracted_data:
        print("First extraction result for paste-and-check:")
        print(extracted_data[0])
    
    return extracted_data

# Run fixed version
if __name__ == "__main__":
    result_fixed = main_weather_extraction_fixed()